In [ ]:
 #library and code setup
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!pip install -q pyspark
import pyspark, os
from pyspark import SparkConf, SparkContext
from pyspark.sql import *
os.environ["PYSPARK_PYTHON"]="python3"
os.environ["JAVA_HOME"]="/usr/lib/jvm/java-8-openjdk-amd64/"
os.environ['PYTHONHASHSEED']="0"


try:
  conf = SparkConf().setMaster("local[*]").set("spark.executor.memory", "1g").set("spark.executorEnv.PYTHONHASHSEED","0").set("spark.ui.port", "4050")
  sc = SparkContext(conf = conf)
  spark = SparkSession.builder.getOrCreate()
except ValueError:
  #it's ok if the server is already started
  pass

def dbg(x):
  """ A helper function to print debugging information on RDDs """
  if isinstance(x, pyspark.RDD):
    print([(t[0], list(t[1]) if 
            isinstance(t[1], pyspark.resultiterable.ResultIterable) else t[1])
           if isinstance(t, tuple) else t
           for t in x.take(100)])
  else:
    print(x)

!pip install gdelt
!pip install newspaper3k

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [787 kB]
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelea

Downloading files from countries: MY, BX, ID, RP, SN and mapping the csv files to output the average tone of the first 10 days in each month, over 28 months period (Jan 1st 2020 to Apr 30th 2022)

In [ ]:
sc = spark.sparkContext
malaysia = spark.read.csv("average tone malaysia.csv").rdd
brunei = spark.read.csv("average tone brunei.csv").rdd
indonesia = spark.read.csv("average tone indonesia.csv").rdd
philippines = spark.read.csv("average tone philippines.csv").rdd
singapore = spark.read.csv("average tone singapore.csv").rdd

#28 months in total
filtered_malaysia = malaysia.map(lambda x: x._c7).take(28)
filtered_brunei = brunei.map(lambda x: x._c4).take(28)
filtered_indonesia = indonesia.map(lambda x: x._c5).take(28)
filtered_philippines = philippines.map(lambda x: x._c4).take(28)
filtered_singapore = singapore.map(lambda x: x._c4).take(28)

#changing lists of strings into lists of floats for algorithm processing
for i in range(0, len(filtered_malaysia)):
  filtered_malaysia[i] = float(filtered_malaysia[i])
dbg(filtered_malaysia)

for i in range(0, len(filtered_brunei)):
  filtered_brunei[i] = float(filtered_brunei[i])
dbg(filtered_brunei)

for i in range(0, len(filtered_indonesia)):
  filtered_indonesia[i] = float(filtered_indonesia[i])
dbg(filtered_indonesia)

for i in range(0, len(filtered_philippines)):
  filtered_philippines[i] = float(filtered_philippines[i])
dbg(filtered_philippines)

for i in range(0, len(filtered_singapore)):
  filtered_singapore[i] = float(filtered_singapore[i])
dbg(filtered_singapore)


[-3.16685, -2.94151, -3.32225, -2.74664, -2.41467, -2.18574, -2.08588, -2.35647, -2.09048, -2.11832, -1.65196, -1.4641, -1.86964, -1.2822, -0.98843, -1.42679, -1.88707, -1.73544, -1.73135, -1.66294, -1.02941, -0.65024, -0.88725, -1.18439, -1.58996, -1.18377, -0.88034, -1.07996]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.97494, -1.62551, -1.61417, -1.11755, -0.71116, -2.10023, -2.4443, -0.65877, -1.73374, -2.12699, -2.11373, -2.51565, -1.70778, -1.08792, -0.92118, -1.29163, -0.61947, -1.77112, -1.75934, -1.32687, -1.73568]
[-1.22734, -2.90591, -2.93214, -2.66097, -2.22166, -2.17838, -1.97064, -1.86686, -1.74859, -1.38174, -0.83428, -0.9163, -0.67271, -1.41072, -0.30265, -0.18069, -0.53646, -0.83384, -1.20847, 0.02363, 0.35628, 0.44167, 1.04154, 0.22395, 0.73123, 0.1297, -0.18093, 1.1023]
[-3.05997, -3.16181, -2.91577, -2.07499, -1.91022, -1.75077, -1.55073, -1.61603, -1.13294, -0.90573, -0.8019, -0.54975, -0.84588, -0.69704, -0.41545, -1.04626, -0.71525, -0.65145, -0.6834, -1.1361, -0.87334

Cosine Similarity between two countries



In [ ]:
import numpy as np

def dot(list1, list2):
  return (sum(a*b for a,b in zip(list1, list2)))

def cosine_similarity(l1, l2):
  return dot(l1,l2) / ((dot(l1,l1) **.5) * (dot(l2,l2) ** .5))

dic = {"MY": filtered_malaysia, "BX": filtered_brunei, "ID": filtered_indonesia, "RP": filtered_philippines, "SN": filtered_singapore}
result = []

#"MY"
test_countries = ['BX', 'ID', 'RP', 'SN']
for i in range(0, len(test_countries)):
  print("MY &", test_countries[i], ":", str(cosine_similarity(dic["MY"], dic[(test_countries[i])])))
  result.append((cosine_similarity(dic["MY"], dic[(test_countries[i])]), "MY & " + test_countries[i]))

#"BX"
test_countries = ['ID', 'RP', 'SN']
for i in range(0, len(test_countries)):
  print("BX &", test_countries[i], ":", str(cosine_similarity(dic["BX"], dic[(test_countries[i])])))
  result.append((cosine_similarity(dic["BX"], dic[(test_countries[i])]), "BX & " + test_countries[i]))


#"ID"
test_countries = ['RP', 'SN']
for i in range(0, len(test_countries)):
  print("ID &", test_countries[i], ":", str(cosine_similarity(dic["ID"], dic[(test_countries[i])])))
  result.append((cosine_similarity(dic["ID"], dic[(test_countries[i])]), "ID & " + test_countries[i]))


#"RP"
print("RP & SN :", str(cosine_similarity(dic["RP"], dic["SN"])))
result.append((cosine_similarity(dic["RP"], dic[(test_countries[i])]), "RP & " + test_countries[i]))


print(sorted(result))

MY & BX : 0.6497956347496313
MY & ID : 0.8035106935860322
MY & RP : 0.9447765960643086
MY & SN : 0.9861158258931872
BX & ID : 0.2537815637327376
BX & RP : 0.42856298510353924
BX & SN : 0.6039167540395614
ID & RP : 0.8406395896804716
ID & SN : 0.8149268274632863
RP & SN : 0.9608286414785763
[(0.2537815637327376, 'BX & ID'), (0.42856298510353924, 'BX & RP'), (0.6039167540395614, 'BX & SN'), (0.6497956347496313, 'MY & BX'), (0.8035106935860322, 'MY & ID'), (0.8149268274632863, 'ID & SN'), (0.8406395896804716, 'ID & RP'), (0.9447765960643086, 'MY & RP'), (0.9608286414785763, 'RP & SN'), (0.9861158258931872, 'MY & SN')]
